Install requirements

In [ ]:
pip install -r requirements.txt    #pyright: ignore

Imports

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Initialising variables

In [7]:
driver = webdriver.ChromiumEdge()
driver.get('https://www.instagram.com/')
username = 'sampleuser'
password = 'samplepass'


#### **1. Logging in**

---

Login to your instagram handle with sample username and password.

In [8]:
driver.get('https://www.instagram.com/')
driver.find_element(By.CSS_SELECTOR, "input[name = 'username']").send_keys(username)    # usename field -> send username
driver.find_element(By.CSS_SELECTOR, "input[name = 'password']").send_keys(password)    # password field -> send password
driver.find_element(By.CSS_SELECTOR,"button[type = 'submit']").click()                  # form submit button -> click

In [9]:
# to close annoying popups after logging in and skip to homepage
(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.cmbtv button')))).click()
(WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div._a9-v button'))))[1].click()

#### **2. Searching**

---

Locate search bar and send query. Returns list of search results

In [16]:
def search(search_query):
    driver.get('https://www.instagram.com/')
    search_bar = WebDriverWait(driver, 10).until(EC.presence_of_element_located(((By.CSS_SELECTOR, 'input[aria-label="Search input"]'))))
    search_bar.clear()
    search_bar.send_keys(search_query)
    return WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(((By.CSS_SELECTOR, 'div._abn- a'))))

Type for "food" in search bar and print all the names of the instagram Handles that are displayed in the list after typing "food".

In [22]:
food_handles = []
for search_result in search('food'):
    search_result = search_result.get_attribute('href').split('/')    # getting profile link of the handles from search result
    try:
        search_result[5]    # if more than 5 parts in the link then assumed link corresponds to explore tag and not user handle
    except IndexError:
        food_handles.append(search_result[3])    # appending the username from proifle link if the link does not corresponds to an explore tag


In [23]:
# exporting found food handles to txt
with open('food_handles.txt', 'w') as text_file:
    for handle in food_handles:
        text_file.write(handle+'\n')


#### **3. Searching and opening profile**

---

Searching and opening profile of 'So Delhi'

In [24]:
search('So Delhi')[0].click()


#### **4. Follow/unfollow given handle**

---

Follow and unfollow handle of 'So Delhi'

In [25]:
def open_profile(profile_name):
    driver.get('https://www.instagram.com/'+profile_name)

In [26]:
# follow
open_profile('sodelhi')
follow_btn = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div._aa_m button')))[1]

if follow_btn.text == 'Follow':
    follow_btn.click()
    print("Now Following")
else:
    print('Already Following')

Now Following


In [27]:
# unfollow 
open_profile('sodelhi')
follow_btn = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div._aa_m button')))[1]

if follow_btn.text == 'Follow':
    print('Already Unfollowed')
else:
    follow_btn.click()
    (WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button._a9--._a9-_')))).click()
    print('Unfollowed')

Unfollowed



#### **5. Like/unlike posts**

---


In [35]:
# open post view of profile from most recent post
def open_post_view(profile_name):
    open_profile(profile_name)
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div._aa_m button')))[1]    # waiting profile page to load completely based on if follow button loaded or not
    driver.find_element(By.CSS_SELECTOR, 'article a').click()    # open most recent post


# like current open post
# pass action as False to dislike instead
def like(action = True):
    try:
        like_btn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section._aamu._aaz9 button._abl-')))
    except:
        return 'Post view not open'
    if like_btn.find_element(By.CSS_SELECTOR, 'svg').get_attribute('aria-label') == 'Like':
        if action:
            like_btn.click()
            return 'liked'
        else:
            return 'already disliked'
    else:
        if action:
            return 'already liked'
        else:
            like_btn.click()
            return 'disliked'
            

# go to next post
def next_post():
    try:
        driver.find_element(By.CSS_SELECTOR, 'div._aaqg._aaqh button').click()
    except:
        return 'Post view not open!'

In [37]:
# Liking top 30 posts of 'So Delhi'
open_post_view('sodelhi')

for i in range(30):
    print(i+1, like())
    next_post()

1 liked
2 liked
3 liked
4 liked
5 liked
6 liked
7 liked
8 liked
9 liked
10 liked
11 liked
12 liked
13 liked
14 liked
15 liked
16 liked
17 liked
18 liked
19 liked
20 liked
21 liked
22 liked
23 liked
24 liked
25 liked
26 liked
27 liked
28 liked
29 liked
30 liked


In [38]:
# Disiking top 30 posts of 'So Delhi'
open_post_view('sodelhi')

for i in range(30):
    print(i+1, like(False))
    next_post()

1 already disliked
2 already disliked
3 disliked
4 already disliked
5 disliked
6 disliked
7 disliked
8 disliked
9 already disliked
10 disliked
11 disliked
12 disliked
13 disliked
14 disliked
15 disliked
16 disliked
17 disliked
18 disliked
19 disliked
20 disliked
21 disliked
22 disliked
23 disliked
24 disliked
25 disliked
26 disliked
27 disliked
28 disliked
29 disliked
30 disliked



#### **6. Extract Followers**

---

Extract usernames of first 500 followers from given profile

In [41]:
# pass alt as True to get following list instead of followers
def find_followers(profile_name, alt = False):
    open_profile(profile_name)
    (WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'ul[class = "_aa_7"] a'))))[int(alt)].click()    # wait till followers button load then click
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div._aano li')))    # wait till initial followers list loads
    except:
        return []    # if follower list doesn't loads after 10 sec, assumed no followers

    scroll_tries = 0
    current_height = 0    # initialising page height
    print("Loading users...")
    while True:
        # collecting all presently loaded followers
        followers_list = driver.find_elements(By.CSS_SELECTOR, 'div._aano li')
        followers_count = len(followers_list)

        # loading statistics
        if followers_count < 500:
            print(followers_count, 'users loaded...')

        # return final list of follower usernames if more than 500 followers loaded or cannot load more followers
        if followers_count >= 500 or scroll_tries == 3:
            print("Loading complete!")
            return [(follower.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')).split('/')[-2] for follower in followers_list[:500]]

        # scroll
        prev_height = current_height
        current_height = driver.execute_script('return document.querySelector("._aano").scrollHeight;')
        driver.execute_script('document.querySelector("._aano").scrollTo(0,arguments[0]);', current_height)
        time.sleep(1)

        # if cannot scroll more/all list items loaded.
        # sometimes, also executed when scroll is not successfull due to page interrupts or long loading times.
        # so, tries scrolling at least three times before returning the final list
        if prev_height == current_height:
            scroll_tries += 1

In [42]:
# extracting required followers/following
print("Finding followers -> foodtalkindia")
followers_foodtalkindia = find_followers('foodtalkindia')

print("\nFinding followers -> sodelhi")
followers_sodelhi = find_followers('sodelhi')

print("\nFinding own followers")
my_followers = find_followers(username)

print("\nFinding own followings")
my_followings = find_followers(username, True)

Finding followers -> foodtalkindia
Loading users...
12 users loaded...
13 users loaded...
24 users loaded...
25 users loaded...
36 users loaded...
48 users loaded...
59 users loaded...
60 users loaded...
71 users loaded...
72 users loaded...
83 users loaded...
95 users loaded...
96 users loaded...
107 users loaded...
118 users loaded...
130 users loaded...
142 users loaded...
143 users loaded...
153 users loaded...
165 users loaded...
166 users loaded...
177 users loaded...
178 users loaded...
189 users loaded...
190 users loaded...
201 users loaded...
202 users loaded...
212 users loaded...
213 users loaded...
224 users loaded...
225 users loaded...
234 users loaded...
246 users loaded...
247 users loaded...
270 users loaded...
271 users loaded...
281 users loaded...
282 users loaded...
294 users loaded...
305 users loaded...
306 users loaded...
317 users loaded...
327 users loaded...
328 users loaded...
339 users loaded...
340 users loaded...
350 users loaded...
351 users loaded...
3

In [45]:
# exporting scrapped follower data to txt files
with open('followers_foodtalkindia.txt', 'w') as text_file:
    for follower in followers_foodtalkindia:
        text_file.write(follower+'\n')

with open('followers_sodelhi.txt', 'w') as text_file:
    for follower in followers_sodelhi:
        text_file.write(follower+'\n')

In [46]:
# printing required result
for user in my_followings:
    if user in followers_foodtalkindia and user not in my_followers:
        print(user)

cloudkitchensmumbai
trigartaofficial
the_magic_spatula_
thecheesecottage.tangmarg
getwhippedindia



#### **7. Check Story**

---

Check story of given profile and print possible scenarios approriately

In [50]:
def check_story(profile_name):
    open_profile(profile_name)
    time.sleep(5)    # assumed profile pic loads in 5 seconds, no better way to condition for now mv as same class exits in amlomst every other page without specific ids
    profile_pic = driver.find_element(By.CSS_SELECTOR, 'div._aarf')
    pic_height = profile_pic.find_element(By.CSS_SELECTOR, 'canvas').get_attribute('height')
    story_disabled =  profile_pic.get_attribute('aria-disabled')
    
    if story_disabled == 'true':
        return 'User has no story'

    else:
        # By observation
        # For seen story pic_height is 208
        # For unseen story pic_height is 210
        # So without much complexities, just conditioned based on this
        if pic_height == '208':
            return 'You have already seen the story'
        if pic_height == '210':
            profile_pic.click()
            return 'Viewing story'

In [51]:
# check out the story of coding.ninjas
check_story('coding.ninjas')

'You have already seen the story'